Hey, I'm Jobert Gutierrez and hereafter you'll find the logic and code used to answer the first assignment in the program Data Engineering Zoomcamp offered by Data Talks Club.

# __Module 1 Homework: Docker & SQL__

When your solution has SQL or shell commands and not code (e.g. python files) file format, include them directly in the README file of your repository.

### __Question 1. Understanding docker first run__
Run docker with the `python:3.12.8` image in an interactive mode, use the entrypoint bash.

What's the version of `pip` in the image?

--24.3.1 <br>
--24.2.1 <br>
--23.3.1 <br>
--23.2.1 <br>

### Answer: 
In the terminal, I coded the command `docker run -it --entrypoint=bash python:3.12.8` to build a new container for the specified python version. Once inside the container, I ran the command `pip list` obtaining a list for all packages already installed. Then, I got the version `24.3.1`.

![q1](Q1.png "Solution 1")

### __Question 2. Understanding Docker networking and docker-compose__
Given the following `docker-compose.yaml`, what is the `hostname` and `port` that pgadmin should use to connect to the postgres database?

```
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

-- postgres:5433 <br>
-- localhost:5432 <br>
-- db:5433 <br>
-- postgres:5432 <br>
-- db:5432

If there are more than one answers, select only one of them

### Answer: 
According to the docker-compose documentation is __postgres:5433__.

## __Prepare Postgres__
Run Postgres and load data as shown in the videos We'll use the green taxi trips from October 2019:

wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

You will also need the dataset with zones:

wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

Download this data and put it into Postgres.

You can use the code from the course. It's up to you whether you want to use Jupyter or a python script.

### __Question 3. Trip Segmentation Count__
During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

- Up to 1 mile
- In between 1 (exclusive) and 3 miles (inclusive),
- In between 3 (exclusive) and 7 miles (inclusive),
- In between 7 (exclusive) and 10 miles (inclusive),
- Over 10 miles

Answers: 

- 104,802; 197,670; 110,612; 27,831; 35,281 <br>
- 104,802; 198,924; 109,603; 27,678; 35,189 <br>
- 104,793; 201,407; 110,612; 27,831; 35,281 <br>
- 104,793; 202,661; 109,603; 27,678; 35,189 <br>
- 104,838; 199,013; 109,645; 27,688; 35,202


In [ ]:
URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"

winpty docker run -it \
  --network=data_projects_default \
  taxi_green:v01 \
    --user=root \
    --password=root \
    --host=pg-database \
    --port=5432 \
    --db=nyc_taxi \
    --table_name=green_taxi_trips \
    --url=${URL}

and:

In [ ]:
URL="wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

winpty docker run -it \
  --network=data_projects_default \
  zones:v01 \
    --user=root \
    --password=root \
    --host=pg-database \
    --port=5432 \
    --db=nyc_taxi \
    --table_name=zones_table \
    --url=${URL}

The we can count the trips into pg-admin using the following postgresql snippet:

In [ ]:
select count(1) from green_taxi_trips
where date(lpep_pickup_datetime)='2019-09-18'
and date(lpep_dropoff_datetime)='2019-09-18'

### Answer:
Hence, we get an amount of __15612 trips__.

### __Question 4. Largest trip for each day__
Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every day, we only care about one single trip with the longest distance.

2019-10-11 <br>
2019-10-24 <br>
2019-10-26 <br>
2019-10-31

we write the code as follows:

In [ ]:
select 
	date(lpep_pickup_datetime) as pick_up_day,
	max(trip_distance) as biggest_distance
from green_taxi_trips
group by date(lpep_pickup_datetime)
order by biggest_distance desc 
limit 3

### Answer:
The largest trip during septembre 2019 was performed in __2019-09-26__.

### __Question 5. Three biggest pickup zones__
Which were the top pickup locations with over 13,000 in `total_amount` (across all trips) for 2019-10-18?

Consider only `lpep_pickup_datetime` when filtering by date.

- East Harlem North, East Harlem South, Morningside Heights <br>
- East Harlem North, Morningside Heights <br>
- Morningside Heights, Astoria Park, East Harlem South <br>
- Bedford, East Harlem North, Astoria Park

Using the following code:

In [ ]:
select 
	public.zones_table."Borough",
	sum(public.green_taxi_trips.total_amount) as sum_amount
from 
	public.zones_table 
left join public.green_taxi_trips
on public.zones_table."LocationID" = public.green_taxi_trips."PULocationID"
where
	date(public.green_taxi_trips.lpep_pickup_datetime)='2019-09-18'
group by 
	public.zones_table."Borough"
order by 
	sum_amount desc

### Answer:
Running the snippet above we identify that the 3 pick up Boroughs that had a sum of total_amount superior to 50000 are __"Brooklyn" "Manhattan" "Queens"__.

### __Question 6. Largest tip__
For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

Note: it's `tip` , not `trip`

We need the name of the zone, not the ID.

- Yorkville West <br>
- JFK Airport <br>
- East Harlem North <br>
- East Harlem South

Using the following code:

In [ ]:
select 
	z2."Zone",
	max(public.green_taxi_trips.tip_amount) as tips
from 
	public.green_taxi_trips
left join public.zones_table z
on z."LocationID" = public.green_taxi_trips."PULocationID"
inner join public.zones_table z2
on z2."LocationID" = public.green_taxi_trips."DOLocationID"
where 
	z."Zone" = 'Astoria'
group by 
	z2."Zone"
order by 
	tips desc
limit 3

### Answer:
We obtain that __"JFK Airport" was the drop off zone that had the largest tip__.

## __Terraform__
In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. Copy the files from the course repo here to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.

### __Question 7. Creating Resources__
Which of the following sequences, respectively, describes the workflow for:

1. Downloading the provider plugins and setting up backend,
2. Generating proposed changes and auto-executing the plan
3. Remove all resources managed by terraform`

Answers:

- terraform import, terraform apply -y, terraform destroy
- teraform init, terraform plan -auto-apply, terraform rm
- terraform init, terraform run -auto-approve, terraform destroy
- terraform init, terraform apply -auto-approve, terraform destroy
- terraform import, terraform apply -y, terraform rm

### Answer:

The output of the command _terraform apply_ is:

Plan: 2 to add, 0 to change, 0 to destroy.

Do you want to perform these actions?
  Terraform will perform the actions described above.
  Only 'yes' will be accepted to approve.

  Enter a value: yes

google_bigquery_dataset.demo_dataset: Creating...
google_storage_bucket.demo-bucket: Creating...
google_bigquery_dataset.demo_dataset: Creation complete after 1s [id=projects/data-taxi-1/datasets/demo_dataset]
google_storage_bucket.demo-bucket: Creation complete after 1s [id=terraform-taxi-data-1]

Apply complete! Resources: 2 added, 0 changed, 0 destroyed.